# Package 설치

In [3]:
%pip install transformers tokenizers datasets accelerate sentencepiece pillow  timm -qU

Note: you may need to restart the kernel to use updated packages.


# Hugging Face Pipeline을 이용한 모델 활용

- Pipeline은 Transformers 라이브러리의 가장 기본적인 객체로, **전처리 - 추론 -> 후처리** 로 이어지는 일련의 과정을 자동화하여 손쉽게 모델을 사용할 수 있게 해준다.
- Task에 따라 다양한 Pipeline 클래스를 제공하며 `pipeline` 함수를 이용해 쉽게 생성할 수 있다.
- **task만 지정**해서 기본 제공 모델과 토크나이저를 사용하거나 **직접 모델과 토크나이저를 지정**해 생성할 수 있다.
- https://huggingface.co/docs/transformers/pipeline_tutorial

![huggingface_pipeline.png](figures/huggingface_pipeline.png)

## 지원하는 주요 태스크
- https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline.task
### 자연어 처리 태스크
- **text-classification**: 텍스트 분류
- **text-generation**: 텍스트 생성
- **translation**: 번역
- **summarization**: 요약
- **question-answering**: 질의응답
- **fill-mask**: 마스크 토큰 채우기
- **token-classification**: 개체명 인식, Pos tagging 같이 개별 토큰에 대한 분류
- **feature-extraction**: 특징 추출(context vector)

### 영상 처리 태스크
- **image-classification**: 이미지 분류
- **object-detection**
  -  객체 검출 (Object Detection)
  -  이미지 안에서 객체들의 위치와 class를 찾아내는 작업
- **image-segmentation**
  -  이미지 세분화 (Image Segmentation)
  -  이미지를 픽셀 단위로 분할하여 각 픽셀이 어떤 객체에 속하는지 분류하는 작업

## 모델 검색
![huggingface_model_search.png](figures/huggingface_model_search.png)



## pipeline 함수
- 주요파라미터
  - **task:** 수행하려는 작업의 유형을 문자열로 지정한다.
  - **model:**
    - 사용할 사전 학습된 모델의 이름 또는 경로를 지정한다. 
    - 모델이름(ID)은 `[모델소유자이름]/[모델이름]` 형식이다. Hugging Face에서 제공하는 모델의 경우는 `모델소유자이름`이 생략되어 있다. (ex: "google/gemma-2-2b", "gpt2")
    - 모델을 명시적으로 지정하지 않으면, **task에 맞는 기본 모델이 로드**된다.
  - **tokenizer:** 자연어 task에서 사용할 토크나이저를 지정한다. 생략하면 모델과 같이 제공되는(model과 이름이 같은 토크나이저) 토크나이저를 사용한다.
  - **framework:** 사용할 딥러닝 프레임워크를 지정한다. 'pt'는 PyTorch(Default), 'tf'는 TensorFlow를 지정한다.
  - **device:** Pipeline 모델을 실행할 디바이스를 지정한다. 문자열로 `"cpu", "cuda:1", "mps"`, 또는 GPU 번호를 정수로 지정한다. 
  - **revision:** 모델의 특정 버전을 지정할 때 사용한다.
  - **trust_remote_code:** hub 모델을 직접 다운 받는 것이 아니라 모델을 다운 받는 **코드**를 다운 받아 local에서 실행하는 경우 코드를 실행할 수있게 할 지 여부. (bool)
  - **use_fast:** 
    - 빠른 토크나이저를 사용할지 여부를 지정합니다. 기본값은 True입니다.
    - 빠른 토크나이저는 `Rust` 언어로 구현되어 속도가 빠르다. 단 모든 모델에 대해 지원하지 않는다. 지원하지 않을 경우 `use_fast=True`로 설정해도 일반 토크나이저가 사용된다.

## Task 별 pipeline 실습

### 텍스트 분류

In [1]:
from transformers import pipeline

In [2]:
# 모델과 토크나이저를 로딩해서 Pipeline을 생성
# 모델, 토크나이저를 생략 -> task에 맞는 모델과 토크나이저를 사용한다.
pipe = pipeline(task="text-classification", framework="pt")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to

In [3]:
result = pipe("I am very happy.")

In [4]:
result

[{'label': 'POSITIVE', 'score': 0.9998763799667358}]

In [5]:
result = pipe("I am very unhappy.")

In [6]:
result

[{'label': 'NEGATIVE', 'score': 0.9997710585594177}]

raw -> || 토큰화 -> 추론 -> 후처리 -> || 최종 예측 결과   
|| 안에 있는 과정을 **파이프라인이 자동으로 해준다.**

In [7]:
data = [ 
    "The project was completed successfully.", 
    "She always brings positive energy to the team.", 
    "I am confident that we will achieve our goals.",
    "The results were not as expected.", 
    "He struggled to meet the deadline.", 
    "The client was dissatisfied with the final product." 
]
result_list = pipe(data)

In [8]:
result_list

[{'label': 'POSITIVE', 'score': 0.9998227953910828},
 {'label': 'POSITIVE', 'score': 0.9998812675476074},
 {'label': 'POSITIVE', 'score': 0.9998470544815063},
 {'label': 'NEGATIVE', 'score': 0.9978100657463074},
 {'label': 'NEGATIVE', 'score': 0.99960857629776},
 {'label': 'NEGATIVE', 'score': 0.9996129870414734}]

In [ ]:
# 특정 모델을 지정해서 사용.
model="distilbert-base-uncased-finetuned-sst-2-english" 
# hugging face에 등록된 모델 id
# 모델 ID 형식: 모델소유자ID/모델ID, 모델소유자ID가 생략된 경우: huggingface에서 만든 자체 모델.
pipe = pipeline(task='text-classification', 
                model=model, # 사용할 모델 지정. hf의 모델 id, 로컬에 저장된 모델저장파일경로.
                tokenizer=model # 사용할 토크나이저를 지정. hf의 토크나이저 id, 로컬에 저장된 토크나이저 파일 경로
                                # 토크나이저의 ID가 모델 ID와 같을 경우 생략.
                )

c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regula

In [11]:
pipe(data)

[{'label': 'POSITIVE', 'score': 0.9998227953910828},
 {'label': 'POSITIVE', 'score': 0.9998812675476074},
 {'label': 'POSITIVE', 'score': 0.9998470544815063},
 {'label': 'NEGATIVE', 'score': 0.9978100657463074},
 {'label': 'NEGATIVE', 'score': 0.99960857629776},
 {'label': 'NEGATIVE', 'score': 0.9996129870414734}]

In [12]:
kor_texts = [
    "이 영화 정말 재미있어요!",
    "서비스가 별로였어요.",
    "제품 품질이 우수합니다.",
    "따듯하고 부드럽고 제품은 너무 좋습니다. 그런데 배송이 너무 늦네요."  # 애매한 것 0.56 정도 나오네.
]

In [13]:
pipe(kor_texts)

[{'label': 'POSITIVE', 'score': 0.9855567812919617},
 {'label': 'POSITIVE', 'score': 0.74257892370224},
 {'label': 'POSITIVE', 'score': 0.6555734872817993},
 {'label': 'NEGATIVE', 'score': 0.5247904658317566}]

In [14]:
model = 'Copycats/koelectra-base-v3-generalized-sentiment-analysis' 
pipe = pipeline(task="text-classification", model=model)
result_list=pipe(kor_texts)

c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--Copycats--koelectra-base-v3-generalized-sentiment-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [15]:
result_list
# 1: 긍정, 0: 부정정

[{'label': '1', 'score': 0.9897311329841614},
 {'label': '0', 'score': 0.9969298243522644},
 {'label': '1', 'score': 0.9640170335769653},
 {'label': '0', 'score': 0.5669130682945251}]

In [16]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--tabularisai--multilingual-sentiment-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [17]:
pipe("오늘은 날씨가 좋아요 햇살이 눈부셔요")

[{'label': 'Very Positive', 'score': 0.49326080083847046}]

### 제로샷 분류
- 제로샷(Zero-shot)은 각 개별 작업에 대한 특정 교육 없이 작업을 수행할 수 있는 task다.
- 입력 텍스트와 함께 클래스 레이블을 제공하면 분류 작업을 한다.
- 모델은  `task`에서 `Zero-Shot` 으로 시작하는 task를 선택하여 검색한다.

In [18]:
model = "facebook/bart-large-mnli"

text = ["Python is a programming language.", 
        "I love soccer", 
        "The stock price rose slightly today."]

labels = ["IT", "Sports"]

In [20]:
pipe = pipeline(task="zero-shot-classification", model=model)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu


In [21]:
result = pipe(text, candidate_labels=labels)

### 텍스트 생성

In [24]:
pipe = pipeline(task="text-generation")

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--openai-community--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.

In [25]:
start_text = "Today weather"

In [26]:
sent = pipe(start_text)
sent

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Today weather conditions are better than expected.\n\n"The temperatures are really good and I\'m hoping the wind will be strong enough to cause a strong gust," he said.\n\nTests indicate the wind gust is about 20 kilometres per hour.\n\nWind gusts in Vancouver are only expected to be about 10 kilometres per hour, but that could change as the weather continues to change, said Greg O\'Brien, an adjunct professor at McMaster University.\n\nMost of the rainwater is coming from Lake Ontario, which is just north of Vancouver, O\'Brien said.\n\n"The water in Lake Ontario is going to be mixed with the water coming from the city, and if it\'s too mixed, it\'s going to get pumped into the lake," he said.\n\nThe storm has been on the move for more than a year now.\n\nThe Canadian Coast Guard, Maritime and Natural Resources Canada, the National Weather Service, the National Weather Service and the provincial Office of National Oceanic and Atmospheric Administration all said th

In [27]:
pipe(["I am", "Python is", "LLM is"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[[{'generated_text': "I am not sure that I will be able to handle this. I am doing the best I can to support and be able to support my daughter and my wife. I have a daughter who has to do with her father. I can't go through what I have already endured. I do not know how to cope with that. I do not want to have to deal with that. I am trying to come to terms with that just as hard as I can. I want to be able to go to school and I want to go to college. But because I am very young and I am struggling with these things, I have not been able to do it in my life. I have struggled with this for years and years. I have struggled with anxiety and depression. I have struggled with depression and anxiety and I have struggled with anxiety and depression and anxiety and depression. I have struggled with all of these things and I have struggled with depression and anxiety and anxiety and I have struggled with anxiety and depression. I have struggled with depression and anxiety and anxiety. I have 

In [29]:
pipe("난 어제") # 한글 학습 안 해서 이상하게 나옴

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '난 어제 나나 서지서 나야로스오이 있사람 안는도 이으를 첡요 아를기지 반미를 가들고 발국이 부들서 있사람 그바가 이성하는 아를기 예부 가 예부 가자이 이성하는 가들고 있사람 그바가 이성하는 가들고 있사람 그바가 이'}]

In [31]:
model_id = 'Qwen/Qwen3-0.6B'
pipe = pipeline(task="text-generation", model=model_id)

c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--Qwen--Qwen3-0.6B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better per

In [ ]:
pipe("나는 어제제")

In [32]:
msg = [
    {"role": "user", "content": "LLM에 대해서 설명해줘."}
]
result = pipe(msg)

In [33]:
result

[{'generated_text': [{'role': 'user', 'content': 'LLM에 대해서 설명해줘.'},
   {'role': 'assistant',
    'content': "<think>\nOkay, the user wants an explanation of LLMs. Let me start by recalling what I know about Large Language Models. They are big language models, right? So I need to define them briefly. Maybe mention that they are trained on huge amounts of text data. Then, explain their capabilities—like understanding and generating text, answering questions, etc. It's important to highlight their role in various applications. I should also mention that they are not human but have some human-like characteristics. Oh, and maybe touch on the training process. They use massive datasets, training algorithms, and optimization techniques. Make sure the explanation is clear and covers key points without getting too technical. Let me check if there's any other aspect I should include, like limitations or future possibilities. Keep the language simple and conversational. Alright, that should cover

In [36]:
result[0]["generated_text"][0] # user 입력(query)

{'role': 'user', 'content': 'LLM에 대해서 설명해줘.'}

In [37]:
result[0]["generated_text"][1] # AI 답변

{'role': 'assistant',
 'content': "<think>\nOkay, the user wants an explanation of LLMs. Let me start by recalling what I know about Large Language Models. They are big language models, right? So I need to define them briefly. Maybe mention that they are trained on huge amounts of text data. Then, explain their capabilities—like understanding and generating text, answering questions, etc. It's important to highlight their role in various applications. I should also mention that they are not human but have some human-like characteristics. Oh, and maybe touch on the training process. They use massive datasets, training algorithms, and optimization techniques. Make sure the explanation is clear and covers key points without getting too technical. Let me check if there's any other aspect I should include, like limitations or future possibilities. Keep the language simple and conversational. Alright, that should cover the basics.\n</think>\n\nLLM은 **대형 언어 모델**을 의미합니다. 이 모델은 매우 큰 텍스트 데이터를 학습

### 마스크 채우기

중간에 들어올 단어를 예측한다!   
- 다음 단어 생성 - GPT
- 중간 단어 채우기 - bert 모델

In [38]:
text = "I'm going to <mask> because <mask> am hurt."
model="distilroberta-base"

In [39]:
pipe = pipeline(task="fill-mask", model=model)
result = pipe(text, top_k=2) # <mask>에 들어갈 확률이 가장 높은 단어 2개(top_k=2)를 찾는다.

c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better p

In [40]:
result

[[{'score': 0.265201210975647,
   'token': 8930,
   'token_str': ' cry',
   'sequence': "<s>I'm going to cry because<mask> am hurt.</s>"},
  {'score': 0.060890499502420425,
   'token': 3581,
   'token_str': ' sleep',
   'sequence': "<s>I'm going to sleep because<mask> am hurt.</s>"}],
 [{'score': 0.9930042624473572,
   'token': 38,
   'token_str': ' I',
   'sequence': "<s>I'm going to<mask> because I am hurt.</s>"},
  {'score': 0.00633621821179986,
   'token': 939,
   'token_str': ' i',
   'sequence': "<s>I'm going to<mask> because i am hurt.</s>"}]]

In [45]:
text = "오늘 밤은 전국이 흐린 가운데 대부분 지역에 [MASK]가 내리겠고, 기온이 내려가면서 점차 [MASK]이 오는 곳이 많겠습니다"
model='beomi/kcbert-base'

In [46]:
pipe = pipeline(task="fill-mask", model=model)
result = pipe(text, top_k=2) # roberta model은 란글을 학습하지 않은 모델델

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [47]:
result

[[{'score': 0.634040355682373,
   'token': 4072,
   'token_str': '##서',
   'sequence': '[CLS] 오늘 밤은 전국이 흐린 가운데 대부분 지역에서 가 내리겠고, 기온이 내려가면서 점차 [MASK] 이 오는 곳이 많겠습니다 [SEP]'},
  {'score': 0.11311633884906769,
   'token': 28206,
   'token_str': '비가',
   'sequence': '[CLS] 오늘 밤은 전국이 흐린 가운데 대부분 지역에 비가 가 내리겠고, 기온이 내려가면서 점차 [MASK] 이 오는 곳이 많겠습니다 [SEP]'}],
 [{'score': 0.10058367997407913,
   'token': 10108,
   'token_str': '바람',
   'sequence': '[CLS] 오늘 밤은 전국이 흐린 가운데 대부분 지역에 [MASK] 가 내리겠고, 기온이 내려가면서 점차 바람 이 오는 곳이 많겠습니다 [SEP]'},
  {'score': 0.0498398020863533,
   'token': 10741,
   'token_str': '영향',
   'sequence': '[CLS] 오늘 밤은 전국이 흐린 가운데 대부분 지역에 [MASK] 가 내리겠고, 기온이 내려가면서 점차 영향 이 오는 곳이 많겠습니다 [SEP]'}]]

### Token별 분류
- task: token-classification 
  - 개체명인식(ner), 품사부착(pos tagging)을 수행하는 task 
  - 개체명 인식은 문장에서 특정한 개체명(예: 사람 이름, 지명, 조직명 등)을 식별하는 task이다. 

In [48]:
text = "My name is Sylvain and I work at Hugging Face in Brooklyn."
model = "dbmdz/bert-large-cased-finetuned-conll03-english"

pipe = pipeline(task='token-classification', model=model)

c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--dbmdz--bert-large-cased-finetuned-conll03-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regu

In [50]:
result = pipe(text)
result

[{'entity': 'I-PER',
  'score': 0.99938285,
  'index': 4,
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity': 'I-PER',
  'score': 0.99815494,
  'index': 5,
  'word': '##yl',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.9959072,
  'index': 6,
  'word': '##va',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.99923277,
  'index': 7,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.9738931,
  'index': 12,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.976115,
  'index': 13,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9887976,
  'index': 14,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9932106,
  'index': 16,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

### 질의 응답
- 문서와 질문을 주면 문서에서 답을 찾아 응답한다.

In [52]:
model = "distilbert-base-cased-distilled-squad"
pipe = pipeline(task="question-answering", model=model)

c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--distilbert-base-cased-distilled-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP dow

In [56]:
question="Where do I work?"
question="Where is Hugging Face?"
context="My name is Sylvain and I work at Hugging Face in Brooklyn"

result = pipe(question=question,  # 질문 
              context=context)    # 답을 찾을 문서

In [57]:
result

{'score': 0.9893267154693604, 'start': 49, 'end': 57, 'answer': 'Brooklyn'}

In [60]:
context = """우리나라 2대 수출 품목인 자동차가 도널드 트럼프 미국 행정부의 관세 여파로 지난달 큰 폭의 수출 감소율을 보이면서 우려가 커지고 있다. 현대차, 기아의 미국 수출 비중이 최대 85%에 이르는 상황에서 자동차 관세 장기화 시 피해는 걷잡을 수 없이 불어날 것이라는 암울한 전망이 나온다.
1일 산업통상자원부가 발표한 5월 수출입 동향에 따르면 지난달 자동차 수출은 작년 동기 대비 4.4% 감소한 62억달러로 집계됐다. 최대 자동차 시장인 미국으로의 수출은 18억4000만달러로 무려 32.0% 급감했다.
4월 미국의 수입산 자동차 25% 관세 부과에 이어 5월부터 일부 자동차 부품에도 25%의 관세가 적용된 결과다. 관세 장기화 시 피해는 더 커질 것이라는 우려가 현실화한 셈이다.
국내 완성차 1·2위 업체인 현대차·기아는 현지 생산 비중을 확대하는 동시에 가격 인상을 검토하고 있다. 관세 여파를 흡수하기 위해서다. 가격 인상이 현실화할 경우 미국 현지 판매는 줄어들 수밖에 없어 수출에는 더 악영향을 미칠 것으로 보인다.
"""

q1 = "현대차 기아의 미국 수출비중은?"
q2 = "자동차 수출이 얼마나 급감했나?"
q3 = "대미 수출 감소에 국내 자동차 업체들의 대응방법은?"

In [59]:
model = "ainize/klue-bert-base-mrc"
pipe = pipeline(task="question-answering", model=model)

c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--ainize--klue-bert-base-mrc. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [61]:
result = pipe(question=[q1, q2, q3], context=context)
result

[{'score': 0.6396177411079407, 'start': 99, 'end': 102, 'answer': '85%'},
 {'score': 0.6323590278625488, 'start': 271, 'end': 276, 'answer': '32.0%'},
 {'score': 0.013693399727344513, 'start': 427, 'end': 433, 'answer': '가격 인상을'}]

### 문서 요약

In [ ]:
model = "eenzeenee/t5-base-korean-summarization"

### 번역

In [ ]:
model = "Helsinki-NLP/opus-mt-fr-en"
text = "Ce cours est produit par Hugging Face."

In [ ]:
model = "Helsinki-NLP/opus-mt-ko-en"

### 이미지를 설명하는 텍스트 생성

In [62]:
url1 = "https://huggingface.co/datasets/Narsil/image_dummy/resolve/main/parrots.png"
url2 = "https://th.bing.com/th?id=ORMS.c526884bbea37c0bb9501f4f83b601e4&pid=Wdp&w=268&h=140&qlt=90&c=1&rs=1&dpr=1&p=0"
url3 = "http://images.cocodataset.org/val2017/000000039769.jpg"

In [64]:
model = "ydshieh/vit-gpt2-coco-en"
pipe = pipeline(task="image-to-text", model=model)
result = pipe([url1, url2, url3])
result

Device set to use cpu


[[{'generated_text': 'two birds are standing next to each other '}],
 [{'generated_text': 'a baseball player is throwing a ball '}],
 [{'generated_text': 'a cat laying on a blanket next to a cat laying on a bed '}]]

### 이미지 분류

In [66]:
url = "https://pds.joongang.co.kr/news/component/htmlphoto_mmdata/202306/25/488f9638-800c-4bac-ad65-82877fbff79b.jpg"

In [65]:
model = "google/vit-base-patch16-224"
pipe = pipeline(task="image-classification", model=model)

c:\Anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--google--vit-base-patch16-224. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. Fo

In [67]:
# 분류 class를 확인
pipe.model.config.id2label

{0: 'tench, Tinca tinca',
 1: 'goldfish, Carassius auratus',
 2: 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 3: 'tiger shark, Galeocerdo cuvieri',
 4: 'hammerhead, hammerhead shark',
 5: 'electric ray, crampfish, numbfish, torpedo',
 6: 'stingray',
 7: 'cock',
 8: 'hen',
 9: 'ostrich, Struthio camelus',
 10: 'brambling, Fringilla montifringilla',
 11: 'goldfinch, Carduelis carduelis',
 12: 'house finch, linnet, Carpodacus mexicanus',
 13: 'junco, snowbird',
 14: 'indigo bunting, indigo finch, indigo bird, Passerina cyanea',
 15: 'robin, American robin, Turdus migratorius',
 16: 'bulbul',
 17: 'jay',
 18: 'magpie',
 19: 'chickadee',
 20: 'water ouzel, dipper',
 21: 'kite',
 22: 'bald eagle, American eagle, Haliaeetus leucocephalus',
 23: 'vulture',
 24: 'great grey owl, great gray owl, Strix nebulosa',
 25: 'European fire salamander, Salamandra salamandra',
 26: 'common newt, Triturus vulgaris',
 27: 'eft',
 28: 'spotted salamander, Ambystoma 

In [69]:
pipe(url, top_k=3)

[{'label': 'Egyptian cat', 'score': 0.8531320691108704},
 {'label': 'tabby, tabby cat', 'score': 0.04750381037592888},
 {'label': 'tiger cat', 'score': 0.034866224974393845}]

### Object Detection

In [72]:
image_path = r"data/image1.jpg"
image_path = r"data/image2.jpg"
image_path = r"data/image3.jpg"

model='facebook/detr-resnet-50'

pipe = pipeline(task="object-detection", model=model)

c:\Anaconda3\envs\dl\Lib\site-packages\torch\nn\modules\module.py:2409: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
c:\Anaconda3\envs\dl\Lib\site-packages\torch\nn\modules\module.py:2409: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
c:\Anaconda3\envs\dl\Lib\site-packages\torch\nn\modules\module.py:2409: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `

In [73]:
pipe(image_path)

[{'score': 0.9956639409065247,
  'label': 'cell phone',
  'box': {'xmin': 96, 'ymin': 165, 'xmax': 136, 'ymax': 236}},
 {'score': 0.9919518232345581,
  'label': 'tv',
  'box': {'xmin': 147, 'ymin': 28, 'xmax': 429, 'ymax': 240}},
 {'score': 0.9975365400314331,
  'label': 'keyboard',
  'box': {'xmin': 108, 'ymin': 251, 'xmax': 358, 'ymax': 304}}]

In [74]:
pipe("data/image1.jpg")

[{'score': 0.9988904595375061,
  'label': 'dog',
  'box': {'xmin': 430, 'ymin': 423, 'xmax': 533, 'ymax': 597}},
 {'score': 0.9998465776443481,
  'label': 'person',
  'box': {'xmin': 531, 'ymin': 158, 'xmax': 673, 'ymax': 581}}]

In [4]:
from transformers import pipeline

In [5]:
pipe = pipeline("text-classification")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [6]:
model = pipe.model
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [8]:
tokenizer = pipe.tokenizer
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased-finetuned-sst-2-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [11]:
pipe("I'm so bad")

[{'label': 'NEGATIVE', 'score': 0.9997639060020447}]